In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from scipy import special

In [3]:
from code.mpmath_series import quad_phi
from code.mpmath_series import series_x_mu_b0_pos, bound_series_x_mu_b0_pos, estimate_N_series_x_mu_b0_pos
from code.mpmath_series import series_x_mu_b0_alternating, bound_series_x_mu_b0_alternating, estimate_N_series_x_mu_b0_alternating

from code.mpmath_series import estimate_N_series_x_mu_b0_pos_fast, estimate_Tk_series_x_mu_b0_pos_fast

In [4]:
from mpmath import *

In [6]:
alpha = 10
mu = 1/5
delta = 1/50
x = 1/3

(x - mu) / delta

omega = np.sqrt((x-mu)**2 + delta**2)
alpha * omega, ((x-mu)**2  / omega * alpha), omega**2

(1.3482498944104453, 1.3185818038243964, 0.018177777777777772)

### 1. Integral

In [7]:
ref = quad_phi(x, alpha, mu, delta, digits=100)
ref

mpf('0.98948463505387288')

### 2. Series

#### 2.1. Series $|x-\mu| \to 0$ with positive terms

Calculate approximated number of terms to obtain a result with absolute error below epsilon.

In [8]:
estimate_N_series_x_mu_b0_pos(x, alpha, mu, delta)

0.9889363528682975 0.028517205838718713 3.9141384783947195e-30 1.1625162349996398e+28 1358.988652904079


mpf('1493.9612955740629')

In [9]:
bound_series_x_mu_b0_pos(x, alpha, mu, delta, 1494)

2.1723187395684038e-16

In [10]:
ref - series_x_mu_b0_pos(x, alpha, mu, delta, 1494)

mpf('2.1405489126701095e-16')

Exponentially improved expansion

In [11]:
from code.mpmath_series import arg_mpmathify

In [22]:
mp.dps = 50

In [23]:
x, alpha, mu, delta = arg_mpmathify(x, alpha, mu, delta)

omega = sqrt(delta**2 + (x-mu)**2)
z = (x-mu)**2 * alpha / omega

In [24]:
N = 50

s = mp.zero
for k in range(N):
    r = z**k / fac2(2*k + 1)
    q = besselk(k + 1, alpha * omega)
    s += r * q

s

mpf('5.3012652143517160428038122853880067383864620562662141')

In [25]:
s = mp.zero
for k in range(N, 1494):
    r = z**k / fac2(2*k + 1)
    q = besselk(k + 1, alpha * omega)
    s += r * q

s

mpf('1.0642227718258769154772702868360347438988816770564311')

In [16]:
aw = alpha * omega

In [319]:
k = 50
besselk(k + 1, aw)

mpf('8.2460324747166454177175410007302e+123')

In [320]:
quad(lambda t: exp(-t - aw**2 / 4 / t) / t**(k+2), [0, inf]) * 1/2 * (aw/2)**(k+1)

mpf('8.24603247471664541780413374920712e+123')

In [321]:
t = 1
nsum(lambda k: (z*aw/2/t)**k / fac2(2*k + 1), [N, inf])

mpf('1.00614595588498974457352780918697e-183')

In [322]:
2**(N-1/2) * (2*N + 1) / fac2(2*N + 1) * exp(z*aw/4/t) * gammainc(N + 1/2, 0, z*aw/4/t) / sqrt(z*aw/2/t)

mpf('1.00614595588498986183887631659806e-183')

#### Remainder: Integral 1

Remainder integral

In [323]:
M = 2**(N-1/2) * (2*N + 1) / fac2(2*N + 1) * aw / 4 / sqrt(z*aw/2)
A = aw ** 2 / 4 - z*aw / 4

M * quad(lambda t: exp(-t - A / t) / t**(3/2) * gammainc(N + 1/2, 0, z*aw/4/t), [0, inf])

mpf('10.7056754938148087950394221146896')

Expand remainder for large $N$

In [324]:
t = 1
r = z*aw/4/t

r, gammainc(N + 1/2, 0, r)

(mpf('0.00444444444444444247071462288861081'),
 mpf('3.23294815913624708484532164790815e-121'))

In [325]:
nsum(lambda k: r**k/rf(N+1/2, k+1), [0, 2]) * r**(N+1/2) * exp(-r)

mpf('3.23294815913428495018887983589497e-121')

In [326]:
nsum(lambda k: (-r)**k/(N + 1/2 + k) / factorial(k), [0, 2]) * r**(N+1/2)

mpf('3.23294820393404934592642817498735e-121')

expansion 1

In [327]:
z*aw/4

mpf('0.00444444444444444247071462288861081')

In [328]:
s = 0
for k in range(100):
    # s += (z*aw/4)**k/rf(N+1/2, k+1) * quad(lambda t: exp(-t - aw ** 2 / 4 / t) / t**(2 + k + N), [0, inf])
    s += (z/2)**k/rf(N+1/2, k+1) * besselk(N + k + 1, aw)

M * s * (z*aw/4)**(N + 1/2) * (2/ aw)**(N+1)*2

mpf('9.93410773779543633312100060460717')

expasion 2

#### Remainder: Integral 2

In [17]:
k = 5
besselk(k + 1, aw)

mpf('584.343111317633297157891404739087')

In [18]:
gamma(k+3/2) * (2*alpha / omega)**(k+1) / sqrt(pi) * quad(lambda t: cos(omega*t) / (t**2 + alpha**2)**(k+3/2), [0, inf])

mpf('584.343111317633295265369072104294')

In [19]:
t = 1
r = 2*z*alpha / omega / (t**2 + alpha**2)

nsum(lambda k: r**k * gamma(k+3/2) / fac2(2*k + 1), [N, inf]), r

(mpf('5.59004691579523709337754068758911'),
 mpf('1.93662398024643538078719546147381'))

In [20]:
gamma(N + 3 / 2) * 2 * r**N / (2 - r) / fac2(2*N + 1)

mpf('5.59004691579523709337754068758832')

In [26]:
M = gamma(N + 3 / 2) * 2 / fac2(2*N + 1) * (2*alpha / omega) / sqrt(pi) * (2*z*alpha / omega)**N

M * quad(lambda t: cos(omega*t) / (t**2 + alpha**2)**(N + 3/2) / (2 - 2*z*alpha / omega / (t**2 + alpha**2)), [0, inf])

mpf('1.061731439141089536174502076619380817149893346189428')

In [29]:
mp.dps = 50

N = 50

x, alpha, mu, delta = arg_mpmathify(x, alpha, mu, delta)
omega = sqrt(delta**2 + (x-mu)**2)
z = (x-mu)**2 * alpha / omega
aw = alpha * omega

s = mp.zero
for k in range(N):
    r = z**k / fac2(2*k + 1)
    q = besselk(k + 1, alpha * omega)
    s += r * q

s1 = s

M = gamma(N + 3 / 2) * 2 / fac2(2*N + 1) * (2*alpha / omega) / sqrt(pi) * (2*z*alpha / omega)**N
m = 2*z*alpha/omega

s = 0
for k in range(N//2):
    # s += (-1)**k * omega**(2*k) / factorial(2*k) * quad(lambda t: t**(2*k) / (t**2 + alpha**2)**(N + 3/2) / (2 - 2*z*alpha / omega / (t**2 + alpha**2)), [0, inf])
    P = 2**(N-2) * alpha**(2*(k - N - 1)) / sqrt(pi) / fac2(2*N - 1) * gamma(N + 1 - k) * gamma(k - 1/2) * hyp2f1(1, N + 1 - k, 3/2 - k, 1 - m/(2*alpha**2))
    Q = 2 **(N - 1 - k) * (2 * alpha**2 - m)**(k - 1/2) * pi * sec(k * pi) / (m **(N + 1/2))
    s += (-1)**k * omega**(2*k) / factorial(2*k) * (P + Q)
s2 = M * s

S = s1 + s2

S * delta * exp(delta * alpha) / pi  * (x-mu)*alpha / omega  + 1/2

mpf('0.98948463505387288294106554208546797858441455182765626')

In [28]:
ref

mpf('0.9894846350538728829410655420854679785844145518276833')

In [30]:
1 - m/(2*alpha**2)

mpf('0.022004889975550132702466291955584513955245880034890743')

In [370]:
N = 50
k = 2
quad(lambda t: t**(2*k) / (t**2 + alpha**2)**(N + 3/2) / (2 - 2*z*alpha / omega / (t**2 + alpha**2)), [0, inf])

mpf('0.000321769096303465513573118498506636')

In [371]:
m = 2*z*alpha/omega
m, 2*alpha**2

(mpf('1.9559902200488997345950674160888'), mpf('2.0'))

In [372]:
1 - m/(2*alpha**2)

mpf('0.0220048899755501327024662919556017')

In [373]:
P = 2**(N-2) * alpha**(2*(k - N - 1)) / sqrt(pi) / fac2(2*N - 1) * gamma(N + 1 - k) * gamma(k - 1/2) * hyp2f1(1, N + 1 - k, 3/2 - k, 1 - m/(2*alpha**2))
Q = 2 **(N - 1 - k) * (2 * alpha**2 - m)**(k - 1/2) * pi * sec(k * pi) / (m **(N + 1/2))

P + Q

mpf('0.00032176909630346551357311849850755')